In [18]:
import requests  # Thư viện để thực hiện các yêu cầu HTTP
from bs4 import BeautifulSoup  # Thư viện để phân tích cú pháp HTML
import csv  # Thư viện để làm việc với tệp CSV

base_url = 'http://203.162.10.109/search?txtKey='
results = []

for i in range(800, 900):
    student_id = f'B21DCCN{i:03}'
    url = f'{base_url}{student_id}'
    response = requests.get(url)
    if response.status_code == 200:  # Kiểm tra nếu yêu cầu thành công
        soup = BeautifulSoup(response.text, 'html.parser')
        rows = soup.find_all('tr')[1:]
        for row in rows:
            name_cell = row.find('td', {'data-label': 'Họ tên'})
            subject_cell = row.find('td', {'data-label': 'Ghi chú'})
            class_cell = row.find('td', {'data-label': 'Lớp'})
            subject_group_cell = row.find('td', {'data-label': 'Nhóm môn học'})
            score_cell = row.find('td', {'data-label': 'Kết quả'})
            if subject_cell and score_cell:  # Kiểm tra nếu có thông tin môn học và điểm
                subject = subject_cell.text.strip()  # Lấy tên môn học
                score = score_cell.text.strip()  # Lấy điểm

                if 'TCP' in subject:
                    # Nếu tìm thấy, lấy thông tin của sinh viên
                    name = name_cell.text.strip() if name_cell else 'Không tìm thấy'  # Tên sinh viên
                    class_name = class_cell.text.strip() if class_cell else 'Không tìm thấy'  # Tên lớp
                    subject_group = subject_group_cell.text.strip() if subject_group_cell else 'Không tìm thấy'  # Nhóm môn học
                    # Lưu thông tin vào danh sách kết quả
                    results.append({'name': name, 'student_id': student_id, 'class': class_name, 'subject_group': subject_group, 'score': score})
                    break  # Ngừng tìm kiếm khi tìm thấy điểm TCP để không lưu thông tin lặp lại

    else:
        print(f'Lỗi khi truy cập {url}: {response.status_code}')  # In thông báo lỗi nếu không truy cập được

# Ghi dữ liệu vào tệp CSV, chỉ ghi những sinh viên có điểm TCP
with open('exam_scores.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['STT', 'Họ tên', 'Mã sinh viên', 'Lớp', 'Nhóm môn học', 'Điểm thi']  # Định nghĩa các cột cho tệp CSV
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)  # Tạo đối tượng viết CSV

    writer.writeheader()  # Ghi tiêu đề vào tệp CSV
    for idx, result in enumerate(results, start=1):  # Duyệt qua từng kết quả
        # Ghi từng dòng dữ liệu vào tệp CSV
        writer.writerow({
            'STT': idx,
            'Họ tên': result['name'],
            'Mã sinh viên': result['student_id'],
            'Lớp': result['class'],
            'Nhóm môn học': result['subject_group'],
            'Điểm thi': result['score']
        })

print("Dữ liệu đã được lưu vào exam_scores.csv")  # Thông báo hoàn thành


Dữ liệu đã được lưu vào exam_scores.csv
